In [ ]:
conda update pandas 

In [ ]:
import os 
import glob
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
from datetime import datetime
import json  
from pandas.io.json import json_normalize




In [ ]:
def extract_text_item(text, start_marker=None, end_marker=None):
    text = text.strip() # clean off \n at the start and end of the text
    
    start_position = None
    end_position = None
    
    if start_marker is not None:
        start_position = text.find(start_marker) + len(start_marker)
        

    if end_marker is not None:
        end_position = text.find(end_marker)
        
    extracted_item = text[start_position:end_position].strip()
    return extracted_item

base_path = "texts"
filenames = glob.glob(os.path.join(base_path,"*.txt"))
filenames

In [ ]:
article_library = []

for filename in filenames:
    with open(filename, 'r') as f:
        whole_text = f.read()

    articles_list = whole_text.split('End of Document\n\n\n')
    useless_endmatter = articles_list.pop(-1) #.pop removes the item at the index specified and return it
    
    for text in articles_list:
        title = extract_text_item(text, start_marker=None, end_marker='\n')
        body = extract_text_item(text, start_marker='Body\n\n\n', end_marker='Load-Date:').strip()
        article_data = {'Title': title, 'Body':body}
        article_library.append(article_data)



In [ ]:
df = pd.DataFrame(article_library)

df = pd.DataFrame(article_library, columns=['Title', 'Body'])


throwaway_df = df.copy()
throwaway_df.index = throwaway_df ['Title']

throwaway_df.index

df.head()

In [ ]:
all_data = pd.DataFrame()
for f in glob.glob ('Excel_MetaData/*.xlsx'):
    df2 = pd.read_excel(f)
    all_data = all_data.append(df2)
    

In [ ]:
writer = pd.ExcelWriter('all_meta_data.xlsx', engine = 'xlsxwriter')
all_data.to_excel(writer, sheet_name = 'Sheet1')
writer.save()


In [ ]:
df2 = df2.drop_duplicates()
print(df2)

df2 = pd.read_excel('all_meta_data.xlsx', sheet_name = 'Sheet1')
print ("Column headings: ")
print(df2.columns)

df2.head()

In [ ]:
df3 = pd.merge(df, df2, on='Title', how='right')

print (df3.columns)

df3.drop(columns=['Unnamed: 0'], axis = 1)


df3.drop_duplicates()


df3.drop(columns=['Unnamed: 0'], axis = 1)

writer = pd.ExcelWriter('df3.xlsx', engine = 'xlsxwriter')
all_data.to_excel(writer, sheet_name = 'Sheet1')
writer.save()

df3.info()

df3.drop(columns = ['Show', 'Word count'])

df3['Published date'] = pd.to_datetime(df3 ['Published date'], errors ='coerce')

writer = pd.ExcelWriter('df3.xlsx', engine = 'xlsxwriter')
all_data.to_excel(writer, sheet_name = 'Sheet1')
writer.save()

df3.drop(columns = ['Unnamed: 0'])

In [ ]:
articles_from_newsapi_2 = pd.read_json('newsapi_2a.json')

df_from_newsapi_2 = pd.DataFrame(articles_from_newsapi_2)
header=None

df_from_newsapi_2

df_from_newsapi_2['publishedAt'] = pd.to_datetime(df_from_newsapi_2['publishedAt'])

df_from_newsapi_2

df_from_newsapi_2.columns

df_from_newsapi_2['source']

In [ ]:
def unnesting(df, explode, axis):
    if axis==1:
        idx = df.index.repeat(df[explode['source']].str.len())
        df1 = pd.concat([
            pd.DataFrame({x: np.concatenate(df[x].values)}) for x in explode], axis=1)
        df1.index = idx

        return df1.join(df.drop(explode, 1), how='left')
    else :
        df1 = pd.concat([
                         pd.DataFrame(df[x].tolist(), index=df.index).add_prefix(x) for x in explode], axis=1)
        return df1.join(df.drop(explode, 1), how='left')


In [ ]:
df_from_newsapi_2 = unnesting(df_from_newsapi_2, ['source'], axis=0)

df_from_newsapi_2 = df_from_newsapi_2.rename(columns={"title": "Title"})

df_from_newsapi_2 = df_from_newsapi_2.drop(columns = ['sourceid'])

df_from_newsapi_2 = df_from_newsapi_2.rename(columns={"content": "Body"})

df_from_newsapi_2 = df_from_newsapi_2.rename(columns={"author": "Author"})

df_from_newsapi_2 = df_from_newsapi_2.rename(columns={"sourcename": "Source"})

df_from_newsapi_2 = df_from_newsapi_2.rename(columns={"description": "Description"})

df_from_newsapi_2 = df_from_newsapi_2.rename(columns={"publishedAt": "Published Date"})

df_from_newsapi_2 = df_from_newsapi_2.drop(columns = ['urlToImage'])

df_from_newsapi_2 = df_from_newsapi_2.rename(columns={"url": "URL"})

df_from_newsapi_2

In [ ]:
#df4 = concatenate df3 to df_from_newsapi_2
frames = [df3, df_from_newsapi_2]
result = pd.concat(frames) 
df4 = result
axis = 1

df4.Body.fillna(df4.Hlead, inplace=True)
del df4['Hlead']

df4 = df4.drop(columns = ["Description"])

df4 = df4.drop(columns = ["Pub-copyright"])

df4 = df4.drop(columns = ["Publication type.1"])

df4 = df4.drop(columns = ["Publication.1"])

df4 = df4.drop(columns = ["Word count"])

df4 = df4.drop(columns = ["Unnamed: 0"])

df4 = df4.drop(columns = ["URL"])

print(df4.columns)

df4 = df4.drop(columns = ["Ticker"])

df4 = df4.drop(columns = ["Show"])

df4 = df4.drop(columns = ["Section"])

df4 = df4.drop(columns = ["Term"])

df4 = df4.drop(columns = ["Headline"])

df4 = df4.drop(columns = ["Length"])

df4 = df4.drop(columns = ["Cite"])

df4 = df4.drop(columns = ["Agg-copyright"])

df4 = df4.drop(columns = ["Publication type"])

print(df4.columns)


writer = pd.ExcelWriter('df4.xlsx', engine = 'xlsxwriter')

writer.save()

df4 = df4.rename(columns={"Published Date": "PublishedDate"})

df4 = df4.rename(columns={"Published date": "Publisheddate"})

df4.head()

print(df4.columns)

writer = pd.ExcelWriter('df4.xlsx', engine = 'xlsxwriter')

writer.save()

cols = df4.columns.tolist()


cols

cols = cols[-1:] + cols[:-1]

cols

df4 = df4[cols]


df4.head()

df4.info()

writer = pd.ExcelWriter('df4.xlsx', engine = 'xlsxwriter')

writer.save()

In [ ]:
base_path = "json"
jsonfilenames = glob.glob(os.path.join(base_path,"*.json"))
jsonfilenames

dfjson = pd.DataFrame()

for file in jsonfilenames:
        data = pd.read_json(file)    
        data.append(data, ignore_index = True) 
    
        dfjson = dfjson.append(data, ignore_index=True)

dfjson.head()

dfjson = unnesting(dfjson, ['source'], axis=0)

In [ ]:
#df5 = concatenate df4 to dfjson
frames = [df4, dfjson]
result = pd.concat(frames) 
df5 = result
axis = 1


df5.head()

df5.Author.fillna(df5.author, inplace=True)
del df5['author']

df5.Title.fillna(df5.title, inplace=True)
del df5['title']

df5.columns

df5 = df5.drop(columns = ['url'])

df5.Source.fillna(df5.sourceid, inplace=True)
del df5['sourceid']

df5.Source.fillna(df5.sourcename, inplace=True)
del df5['sourcename']

df5.Body.fillna(df5.content, inplace=True)
del df5['content']

df5 = df5.drop(columns = ['urlToImage'])

df5.PublishedDate.fillna(df5.publishedAt, inplace=True)
del df5['publishedAt']

df5 = df5.drop(columns = ["description"])

cols = df5.columns.tolist()

cols

cols = cols[-1:] + cols[:-1]

df5 = df5[cols]


print(df5['Source'])


In [ ]:
articles_from_newsapi_may = pd.read_json('df_may_newsapi.json')

articles_from_newsapi_may = pd.DataFrame(articles_from_newsapi_may)
header=None

articles_from_newsapi_may.head()

articles_from_newsapi_may.columns

articles_from_newsapi_may['articles']

articles_from_newsapi_may = articles_from_newsapi_may.explode('articles')

articles_from_newsapi_may = articles_from_newsapi_may.drop(columns = ["status"])

articles_from_newsapi_may = articles_from_newsapi_may.drop(columns = ["totalResults"])

articles_from_newsapi_may.head()

list(articles_from_newsapi_may)

articles_from_newsapi_may = unnesting(articles_from_newsapi_may, ['articles'], axis=0)

articles_from_newsapi_may.head()

articles_from_newsapi_may = articles_from_newsapi_may.drop_duplicates(subset=['articlestitle'])

articles_from_newsapi_may

articles_from_newsapi_may = articles_from_newsapi_may.drop(columns = ["articlesdescription"])

articles_from_newsapi_may = articles_from_newsapi_may.drop(columns = ["articlesurl"])

articles_from_newsapi_may = articles_from_newsapi_may.drop(columns = ["articlesurlToImage"])

articles_from_newsapi_may.head()

articles_from_newsapi_may = articles_from_newsapi_may.rename(columns={"articlesauthor": "Author"})

articles_from_newsapi_may = articles_from_newsapi_may.rename(columns={"articlespublishedAt": "PublishedDate"})

articles_from_newsapi_may = articles_from_newsapi_may.rename(columns={"articlescontent": "Body"})

articles_from_newsapi_may = articles_from_newsapi_may.rename(columns={"articlestitle": "Title"})

articles_from_newsapi_may.head()

articles_from_newsapi_may.reset_index(drop=True)

articles_from_newsapi_may = unnesting(articles_from_newsapi_may, ['articlessource'], axis=0)

articles_from_newsapi_may.head()

In [ ]:
#df6 = concatenate df5 to articles_from_newsapi_may
frames1 = [df5, articles_from_newsapi_may]
result1 = pd.concat(frames1) 
df6 = result1
axis = 1

df6.Source.fillna(df6.articlessourceid, inplace=True)


df6.Source.fillna(df6.articlessourcename, inplace=True)
del df6['articlessourcename']

df6.columns

df6.head()

df6 = df6.drop_duplicates()

df6 = df6.reset_index(drop=True)

df6.Author.fillna(df6.Byline, inplace=True)
del df6['Byline']

df6.Author.fillna(df6.Source, inplace=True)


df6.Author.fillna(df6.Company, inplace=True)


df6.Source.fillna(df6.articlessourceid, inplace=True)
del df6 ['articlessourceid']

df6.Source.fillna(df6.Company, inplace=True)
del df6 ['Company']

df6.Source.fillna(df6.Publication, inplace=True)
del df6 ['Publication']

df6.Publisheddate.fillna(df6.PublishedDate, inplace=True)
del df6['PublishedDate']

df6 = df6.dropna(subset=['Body'], axis='index')


df6.info()

df6 = df6.set_index('Publisheddate', drop=True)

df6.Author.fillna(df6.Source, inplace=True)


df6["Publication location"].fillna("International", inplace = True)

df6 = df6.where((pd.notnull(df6)), None)

df6.info()


writer1 = pd.ExcelWriter('df6.xlsx', engine = 'xlsxwriter')

writer.save()


df6.to_csv('df6.csv')

In [ ]:
print(df6["Countries"].isnull())

In [ ]:
df6['Source']

In [ ]:
to_drop = df6[df6.Source.str.contains("Sophos.com")].index
df6.drop(to_drop , inplace=True)

to_dropb = df6[df6.Source.str.contains("Thehackernews.com")].index
df6.drop(to_dropb , inplace=True)

to_drop1 = df6[df6.Source.str.contains("Schneier.com")].index
df6.drop(to_drop1 , inplace=True)

df6.loc[df6.Source == "Huffpost.com", "Countries"] = "United States"

to_drop2 = df6[df6.Source.str.contains("Youtube.com")].index
df6.drop(to_drop2 , inplace=True)

df6.loc[df6.Source == "Bbc.com", "Countries"] = "United Kingdom"

df6.loc[df6.Source == "bbc-news", "Countries"] = "United Kingdom"

to_drop3 = df6[df6.Source.str.contains("Techmeme.com")].index
df6.drop(to_drop3 , inplace=True)

to_drop4 = df6[df6.Source.str.contains("engadget")].index
df6.drop(to_drop4 , inplace=True)

df6.loc[df6.Source == "Wipo.int", "Countries"] = "Switzerland"

df6.loc[df6.Source == "Forbes.com", "Countries"] = "United States"

df6.loc[df6.Source == "TechCrunch", "Countries"] = "United States"

df6.loc[df6.Source == "techcrunch", "Countries"] = "United States"

df6.loc[df6.Source == "ars-technica", "Countries"] = "United States"

df6.loc[df6.Source == "the-next-web", "Countries"] = "Netherlands"

df6.loc[df6.Source == "Yahoo.com", "Countries"] = "United States"

df6.loc[df6.Source == "business-insider", "Countries"] = "United States"

df6.loc[df6.Source == "Business Insider", "Countries"] = "United States"

df6.loc[df6.Source == "Cnet.com", "Countries"] = "United States"

df6.loc[df6.Source == "Technologyreview.com", "Countries"] = "United States"

to_drop5 = df6[df6.Source.str.contains("Gizmodo.com")].index
df6.drop(to_drop5 , inplace=True)

to_drop6 = df6[df6.Source.str.contains("Gizmodo.com")].index
df6.drop(to_drop6 , inplace=True)

to_drop7 = df6[df6.Source.str.contains("Koinpost.com")].index
df6.drop(to_drop7 , inplace=True)

to_drop8 = df6[df6.Source.str.contains("Slashdot.org")].index
df6.drop(to_drop8 , inplace=True)

df6.loc[df6.Source == "Avclub", "Countries"] = "United States"

df6.loc[df6.Source == "Avclub.com", "Countries"] = "United States"

to_drop9 = df6[df6.Source.str.contains("Lifehacker.com")].index
df6.drop(to_drop9 , inplace=True)

df6.loc[df6.Source == "Wired", "Countries"] = "United States"

df6.loc[df6.Source == "wired", "Countries"] = "United States"

to_drop10 = df6[df6.Source.str.contains("Substack.com")].index
df6.drop(to_drop10 , inplace=True)

to_drop10a = df6[df6.Source.str.contains("Fool.com")].index
df6.drop(to_drop10a , inplace=True)

df6.loc[df6.Source == "the-new-york-times", "Countries"] = "United States"

df6.loc[df6.Source == "al-jazeera-english", "Countries"] = "Qatar"

to_drop11 = df6[df6.Source.str.contains("Davidwalsh.name")].index
df6.drop(to_drop11 , inplace=True)

to_drop11a = df6[df6.Source.str.contains("mashable")].index
df6.drop(to_drop11a , inplace=True)

to_drop12 = df6[df6.Source.str.contains("Readwrite.com")].index
df6.drop(to_drop12 , inplace=True)

to_drop12a = df6[df6.Source.str.contains("Androidcentral.com")].index
df6.drop(to_drop12a , inplace=True)

to_drop13 = df6[df6.Source.str.contains("Marketwatch.com")].index
df6.drop(to_drop13 , inplace=True)

to_drop13a = df6[df6.Source.str.contains("Venturebeat.com")].index
df6.drop(to_drop13a , inplace=True)

to_drop14 = df6[df6.Source.str.contains("Boingboing.net")].index
df6.drop(to_drop14 , inplace=True)

to_drop16 = df6[df6.Source.str.contains("Salon.com")].index
df6.drop(to_drop16 , inplace=True)

to_drop16a = df6[df6.Source.str.contains("Theregister.co.uk")].index
df6.drop(to_drop16a , inplace=True)

df6.drop(df6.index[df6.Source == "Theregister"], inplace=True)

df6.drop(df6.index[df6.Source == "Thehustle.co"], inplace=True)

df6.drop(df6.index[df6.Source == "Androidauthority.com"], inplace=True)

df6.drop(df6.index[df6.Source == "Entrepreneur.com"], inplace=True)

df6 = df6.dropna()

In [ ]:
df6.loc[df6.Source == "The New York Times - International Edition", "Countries"] = "United States"

df6['Source'].replace(['ALTRIA GROUP INC'], "The New York Times - International Edition", inplace=True)

df6['Source'].replace(['FACEBOOK INCPAYPAL HOLDINGS INC'], "The New York Times - International Edition", inplace=True)

df6['Source'].replace(["DUNKIN' BRANDS GROUP INC"], "The New York Times - International Edition", inplace=True)

df6['Source'].replace(["WALL STREET JOURNALFACEBOOK INC"], "The New York Times - International Edition", inplace=True)

df6['Source'].replace(["WALL STREET JOURNALFACEBOOK INC"], "The New York Times - International Edition", inplace=True)

df6['Source'].replace(["UNIVERSITY OF TEXASUS DEPARTMENT OF JUSTICE"], "The New York Times - International Edition", inplace=True)

df6['Source'].replace(["FACEBOOK INCVISA INC"], "The New York Times - International Edition", inplace=True)

df6['Source'].replace(["FACEBOOK INC"], "The New York Times - International Edition", inplace=True)

df6['Source'].replace(["FACEBOOK INC"], "The New York Times - International Edition", inplace=True)

df6['Source'].replace(["FACEBOOK INCEUROPEAN CENTRAL BANK"], "The New York Times - International Edition", inplace=True)

df6['Source'].replace(["DEMOCRATIC NATIONAL COMMITTEE"], "The New York Times - International Edition", inplace=True)

df6['Source'].replace(["PAYPAL HOLDINGS INC"], "The New York Times - International Edition", inplace=True)

df6['Source'].replace(["ContentEngine Think Tank Newswire English"], "libertarianinstitute.org", inplace=True)

df6.loc[df6.Source == "libertarianinstitute.org", "Countries"] = "United States"

df6['Source'].replace(["EASTMAN KODAK COLONG ISLAND ICED TEA CORP"], "The New York Times - International Edition", inplace=True)

df6.loc[df6['Countries'] == "Foreign"]

df6['Countries'].replace(["Foreign"], "United States", inplace=True)

In [ ]:
df6

In [ ]:
df6.loc[df6['Source'] == "vice-news"]